In [ ]:
import requests, time, json
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


with open("results/nerc_datacite_dois.json") as f:
    nerc_datacite_dois = json.load(f)

nerc_datacite_dois_df = pd.DataFrame(nerc_datacite_dois)

print(len(nerc_datacite_dois_df))



5425


In [ ]:
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("https://", HTTPAdapter(max_retries=retries))

def overton_api_request(doi):
    url = "https://app.overton.io/documents.php"
    params ={
        "plain_dois_cited": doi,
        "format": "json",
        "api_key": "3c7b1a-849d90-77f9da",
    }
    r = session.get(url, params=params, timeout=30)
    print(r.status_code, "DOI: ", doi)
    r.raise_for_status()
    return r.json().get('results', [])


In [ ]:
results = []

for doi in nerc_datacite_dois_df.data_doi:
    start = time.time()

    try:
        data = overton_api_request(doi)
        if data:
            results.append(data)
    except requests.RequestException as e:
        print(f"Error on {doi}: {e}")
        continue

    elapsed = time.time() - start
    # enforce at least 1s between requests
    if elapsed < 1:
        time.sleep(1 - elapsed)

In [ ]:
results

In [ ]:
# overton api single request
# publishers_cited=_%3Ax39

url = "https://app.overton.io/documents.php"
params ={
    "format": "json",
    "api_key": "3c7b1a-849d90-77f9da",
    "plain_dois_cited": "10.5285/e0f0bb80-ab44-2739-e053-6c86abc0289c"
}


r = requests.get(url,params=params)

print(r.status_code)
data = r.json()


print(len(data['results']))
data['results'][1]

In [4]:
doi = "10.5285/80887755-1426-4dab-a4a6-250919d5020c"
data = overton_api_request(doi)
len(data)

200 DOI:  10.5285/80887755-1426-4dab-a4a6-250919d5020c


0

In [ ]:
# old - this will take about 90 mins - 5400 DOIs at about 1 per second
results = list()

for doi in nerc_datacite_dois_df.data_doi:
    start = time.time()

    data = overton_api_request(doi)
    if (len(data) > 0):
        results.append(data)

    elapsed = time.time() - start
    print("elapsed:", elapsed)

    # rate limiting to 1 per second
    if elapsed < 1:
        time.sleep(1.05 - elapsed)


    

    
